In [1]:
import arcpy
arcpy.env.workspace = r'C:\Users\ltmsbi\Documents\ArcGIS\Projects\Final_Deployment\Final_Deployment.gdb'

In [4]:
# Create layers per-species and geometry then add names to list for spatial joining


# Create list of n + n*3 files to process, where n is the number of species
ps_records = [] # empty list to append items onto

# Create dictionary of long names
# Values are not used in this code and are only present for human readability
species_fullnames = {
    "phrag": "Phragmites, Unspecified",
    "knot": "Knotweed, Unspecified",
    "wp": "Wild Parsnip",
    "toh": "Tree-of-Heaven (Ailanthus)",
    "pl": "Purple Loosestrife"
}

# Extract only the keys to a list
species_shortnames = list(species_fullnames.keys())


# Add model data layer names to processing list ----------


# Define the threshold used to determine presences
threshold = 'precision'

# Add names to list
for n in range(0,len(species_shortnames)):
    # Add n items for model data 
    ps_records.append('SVI_Project_presences_'+species_shortnames[n]+"_"+threshold)

    
# ----------
    
# Separate out iMap records by species and geometry ----------


# IDs that iMap assigns to the various species of interest
# Used for building queries to select by attribute
jurisdiction_ids = {
    "phrag": 1277,
    "wp": 1182,
    "pl": 1265,
    "toh": 1167,
    "knot": (1074, 1191, 1278, 1479) # Includes Japanese knotweed, giant knotweed, bohemian knotweed, and knotweed species unknown
}

# Define list of geometries in the iMap data
imap_geometries = ["POINT", "LINE", "POLYGON"]

# Define imap records types
imap_record_types = {
    "cnfrm": "", # The "suffix" for confirmed records is blank
    "uncnfrm": "_UNCONFIRMED"
}

# Add n*2*3 + n items for iMap data (accounts for 2 record and 3 geometry types, 
# plus not-detected records) 
for n in range(0,len(species_shortnames)):
    # Set up per-species query for selecting by attribute
    if species_shortnames[n] is "knot":
        # Set initial SQL query
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids["knot"][0])
        # Add more conditions to query
        for ID in jurisdiction_ids["knot"][1:]:
            idClause = idClause + " Or jurisdiction_species_id = " + str(ID)
    else:
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids[species_shortnames[n]]) 
    
    # Copy a per-species subset of not detected polygons
    sel = arcpy.management.SelectLayerByAttribute("NOT_DETECTED_POLYGON", "NEW_SELECTION", idClause)
    imap_nd = "iMap_nd_"+species_shortnames[n]
    arcpy.management.CopyFeatures(sel, imap_nd)
    ps_records.append(imap_nd)
    # Remove variables and selections to save memory
    del sel, imap_nd
    arcpy.management.SelectLayerByAttribute("NOT_DETECTED_POLYGON", "CLEAR_SELECTION")
    
    # Copy a per-species subset for each record and geometry type
    for rt in range(0,len(imap_record_types)):
        for g in range(0,len(imap_geometries)):
            sel = arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[g]+list(imap_record_types.values())[rt], "NEW_SELECTION", idClause)
            imap_subset = "iMap_"+imap_geometries[g].lower()+"_"+list(imap_record_types.keys())[rt]+"_"+species_shortnames[n]
            arcpy.management.CopyFeatures(sel, imap_subset)
            ps_records.append(imap_subset)
            # Remove variables and selections to save memory
            del sel, imap_subset
            arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[g], "CLEAR_SELECTION")
        
        
# ----------

print("Number of layers to process: "+str(len(ps_records)))
print(ps_records)

Number of layers to process: 40
['SVI_Project_presences_phrag_precision', 'SVI_Project_presences_knot_precision', 'SVI_Project_presences_wp_precision', 'SVI_Project_presences_toh_precision', 'SVI_Project_presences_pl_precision', 'iMap_nd_phrag', 'iMap_point_cnfrm_phrag', 'iMap_line_cnfrm_phrag', 'iMap_polygon_cnfrm_phrag', 'iMap_point_uncnfrm_phrag', 'iMap_line_uncnfrm_phrag', 'iMap_polygon_uncnfrm_phrag', 'iMap_nd_knot', 'iMap_point_cnfrm_knot', 'iMap_line_cnfrm_knot', 'iMap_polygon_cnfrm_knot', 'iMap_point_uncnfrm_knot', 'iMap_line_uncnfrm_knot', 'iMap_polygon_uncnfrm_knot', 'iMap_nd_wp', 'iMap_point_cnfrm_wp', 'iMap_line_cnfrm_wp', 'iMap_polygon_cnfrm_wp', 'iMap_point_uncnfrm_wp', 'iMap_line_uncnfrm_wp', 'iMap_polygon_uncnfrm_wp', 'iMap_nd_toh', 'iMap_point_cnfrm_toh', 'iMap_line_cnfrm_toh', 'iMap_polygon_cnfrm_toh', 'iMap_point_uncnfrm_toh', 'iMap_line_uncnfrm_toh', 'iMap_polygon_uncnfrm_toh', 'iMap_nd_pl', 'iMap_point_cnfrm_pl', 'iMap_line_cnfrm_pl', 'iMap_polygon_cnfrm_pl', 'iMap

In [5]:
# Spatial join repeatedly

# Effectively, for iMap and model data, each species, and geometry/record type (if iMap data):
    # Join number of points to the reporting grid created in previous run
    
    
    # Calculate model "not detected" as all model points joined - model species presence

2


In [ ]:
# Build the analysis-ready version (1 layer with numbers for all species)

# Build the viewing-ready version (n per-species layers combined into one)

# For each species:
    # Select records with non-null values for that species
    # Add & calculate common name, jurisdiction id, and op criteria fields
# Merge layers into one for upload into AGOL